In [1]:
from helpers import *
import pickle
import pandas as pd
import torch
import torch.nn.functional as F

In [2]:
def load_model(model="gcn_2", split="train"):
    assert model in ["gcn_2", "gcn_6", "adgpm"]
    assert split in ["train", "goz_v1", "generalized"]
    if split == "generalized":
        nodes = pickle.load(open("../data/splits/goz_v1".format(split), "rb")) +\
                pickle.load(open("../data/splits/train".format(split), "rb"))
    else:
        nodes = pickle.load(open("../data/splits/{}".format(split), "rb"))
    df = pd.read_pickle("../data/models/{}".format(model))
    w  = torch.from_numpy(df.loc[nodes].values[:,:-1]).t().cuda()
    return w
    
def validate(x, w):
    """
    """
    return F.softmax(x.cuda().mm(w), dim=-1)

# Standard ZSL

In [3]:
x_te,y_te,s_te=load_test_set(generalized=False)

500it [00:13, 37.85it/s]


In [4]:
w = load_model(model="gcn_2", split="goz_v1")
out = validate(x_te, w)
pp(topk(out, y_te.cuda()))

 14.09 | 21.66 | 27.34 | 31.79 | 35.12 


In [5]:
w = load_model(model="gcn_6", split="goz_v1")
out = validate(x_te, w)
pp(topk(out, y_te.cuda()))

 9.58 | 15.66 | 20.33 | 24.05 | 27.19 


In [6]:
w = load_model(model="adgpm", split="goz_v1")
out = validate(x_te, w)
pp(topk(out, y_te.cuda()))

 14.10 | 22.21 | 28.12 | 32.52 | 36.03 


# Generalized ZSL

In [7]:
x_te,y_te,s_te=load_test_set(generalized=True, filter_out=False)

/media/tristan/Samsung_T5/workspace/GOZ/benchmark/helpers.py:43: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  x  = df.loc[list(nodes)].fillna(0).values
1500it [00:01, 1004.55it/s]


In [8]:
w = load_model(model="gcn_2", split="generalized")
out = validate(x_te, w)
msk_test  = y_te<500
msk_train = y_te>500

test_accs  = topk(out[msk_test], y_te[msk_test].cuda())
train_accs = topk(out[msk_train], y_te[msk_train].cuda())
h_mean = h_scores(test_accs, train_accs)

pp(h_mean)

 4.96 | 12.83 | 19.82 | 25.44 | 30.35 


In [9]:
w = load_model(model="gcn_6", split="generalized")
out = validate(x_te, w)
msk_test  = y_te<500
msk_train = y_te>500

test_accs  = topk(out[msk_test], y_te[msk_test].cuda())
train_accs = topk(out[msk_train], y_te[msk_train].cuda())
h_mean = h_scores(test_accs, train_accs)

pp(h_mean)

 4.81 | 10.16 | 14.93 | 19.26 | 23.35 


In [10]:
w = load_model(model="adgpm", split="generalized")
out = validate(x_te, w)
msk_test  = y_te<500
msk_train = y_te>500

test_accs  = topk(out[msk_test], y_te[msk_test].cuda())
train_accs = topk(out[msk_train], y_te[msk_train].cuda())
h_mean = h_scores(test_accs, train_accs)

pp(h_mean)

 4.90 | 12.27 | 19.05 | 24.97 | 29.96 
